# Exploratory Data Analysis

This notebook contains tools and examples for data science and engineering using PySpark, Pandas, Polars, Scikit-learn, NumPy, and other libraries.

## Setup and Imports

In [2]:
#remove deprecated packages
# Basic data manipulation and analysis
import numpy as np
import pandas as pd
from ydata_profiling import ProfileReport, compare

## Initialize Spark Session

## Data Loading

In [4]:
# Directory containing Parquet files
data_dir = "/home/manuel-bayona/Documents/crisil2/data/"

# List of Parquet file names
parquet_files = [
    "cubesmart_weekly_20250127_181344.parquet",
    "cubesmart_weekly_20250203_181358.parquet",
    "cubesmart_weekly_20250210_061336.parquet",
    "smartstopselfstorage_weekly_20250127_090106.parquet",
    "smartstopselfstorage_weekly_20250203_090120.parquet",
    "smartstopselfstorage_weekly_20250210_060127.parquet",
    "storagemart_weekly_20250127_090843.parquet",
    "storagemart_weekly_20250203_091401.parquet",
    "storagemart_weekly_20250210_061303.parquet"
]

In [4]:
import os
from collections import defaultdict
# Store schema per file
file_schemas = {}

# Load each file and store its schema
for file in parquet_files:
    file_path = os.path.join(data_dir, file)
    try:
        df = pd.read_parquet(file_path)
        schema = tuple((col, str(dtype)) for col, dtype in df.dtypes.items())
        file_schemas[file] = schema
        print(f"Schema extracted for file: {file}")
    except Exception as e:
        print(f"Failed to load Parquet file {file}: {e}")

# Group files with identical schemas
schema_groups = defaultdict(list)
for file, schema in file_schemas.items():
    schema_groups[schema].append(file)

# Output grouped schemas for manual field mapping
print("\n=== Grouped Files by Schema ===")
for idx, (schema, files) in enumerate(schema_groups.items(), start=1):
    print(f"\nGroup {idx}: {len(files)} file(s)")
    for f in files:
        print(f"  - {f}")
    print("Schema:")
    for col, dtype in schema:
        print(f"  {col}: {dtype}")

Schema extracted for file: cubesmart_weekly_20250127_181344.parquet
Schema extracted for file: cubesmart_weekly_20250203_181358.parquet
Schema extracted for file: cubesmart_weekly_20250210_061336.parquet
Schema extracted for file: smartstopselfstorage_weekly_20250127_090106.parquet
Schema extracted for file: smartstopselfstorage_weekly_20250203_090120.parquet
Schema extracted for file: smartstopselfstorage_weekly_20250210_060127.parquet
Schema extracted for file: storagemart_weekly_20250127_090843.parquet
Schema extracted for file: storagemart_weekly_20250203_091401.parquet
Schema extracted for file: storagemart_weekly_20250210_061303.parquet

=== Grouped Files by Schema ===

Group 1: 6 file(s)
  - cubesmart_weekly_20250127_181344.parquet
  - cubesmart_weekly_20250203_181358.parquet
  - cubesmart_weekly_20250210_061336.parquet
  - storagemart_weekly_20250127_090843.parquet
  - storagemart_weekly_20250203_091401.parquet
  - storagemart_weekly_20250210_061303.parquet
Schema:
  Run_Date: 

# Schema Group Analysis of Parquet Files

This notebook analyzes the schema structures across 9 Parquet files from 3 different retailers. The goal is to identify similarities and differences in schema definitions, which is a critical step before performing unified data transformations or ingestion into a central data platform.

---

## Group Overview

The analysis grouped the files into two schema groups based on exact matches in field names and data types.

### **Group 1** (6 files)
- **Retailers**: `cubesmart`, `storagemart`
- **Files**:
  - `cubesmart_weekly_20250127_181344.parquet`
  - `cubesmart_weekly_20250203_181358.parquet`
  - `cubesmart_weekly_20250210_061336.parquet`
  - `storagemart_weekly_20250127_090843.parquet`
  - `storagemart_weekly_20250203_091401.parquet`
  - `storagemart_weekly_20250210_061303.parquet`

**Schema Fields**:
- `Run_Date`: datetime64[ns]  
- `Retailer`: object  
- `Retailer_Website`: object  
- `Store_Info`: object  
- `Category_Traversal`: object  
- `Brand`: object  
- `Product_Name`: object  
- `Product_ID`: object  
- `Universal_Product_ID`: object  
- `Variant_Info`: object  
- `Size_Or_Quantity`: object  
- `Number_Of_Reviews`: float64  
- `Average_Rating`: object  
- `Original_Price`: object  
- `Sale_Price`: object  
- `Currency`: object  
- `Url`: object  
- `In_Stock`: bool  

---

### **Group 2** (3 files)
- **Retailer**: `smartstopselfstorage`
- **Files**:
  - `smartstopselfstorage_weekly_20250127_090106.parquet`
  - `smartstopselfstorage_weekly_20250203_090120.parquet`
  - `smartstopselfstorage_weekly_20250210_060127.parquet`

**Schema Fields**:
- `Run_Date`: datetime64[ns]  
- `Retailer`: object  
- `Retailer_Website`: object  
- `Store_Info`: object  
- `Category_Traversal`: object  
- `Brand`: object  
- `Product_Name`: object  
- `Product_ID`: object  
- `Universal_Product_ID`: object  
- `Variant_Info`: object  
- `Size_Or_Quantity`: object  
- `Number_Of_Reviews`: float64  
- `Average_Rating`: float64  
- `Original_Price`: float64  
- `Sale_Price`: float64  
- `Currency`: object  
- `Url`: object  
- `In_Stock`: bool  

---

## Key Differences

Although all files share the same field names and structure, the following fields differ in data type:

| Field              | Group 1 Type | Group 2 Type |
|-------------------|--------------|--------------|
| `Average_Rating`  | object       | float64      |
| `Original_Price`  | object       | float64      |
| `Sale_Price`      | object       | float64      |

These differences indicate inconsistencies in how numeric values are stored. Group 1 files likely contain string representations of numbers (possibly with currency symbols or invalid formats), while Group 2 files have clean numeric values.

---

## Conclusion

The schema is structurally aligned across all files but differs in type quality for numeric fields. This should be addressed in preprocessing steps to ensure consistency.

In [5]:
# Define canonical column types
canonical_dtypes = {
    "Run_Date": "datetime64[ns]",
    "Retailer": "string",
    "Retailer_Website": "string",
    "Store_Info": "string",
    "Category_Traversal": "string",
    "Brand": "string",
    "Product_Name": "string",
    "Product_ID": "string",
    "Universal_Product_ID": "string",
    "Variant_Info": "string",
    "Size_Or_Quantity": "string",
    "Number_Of_Reviews": "float64",
    "Average_Rating": "float64",
    "Original_Price": "float64",
    "Sale_Price": "float64",
    "Currency": "string",
    "Url": "string",
    "In_Stock": "boolean"
}

In [6]:
def load_and_normalize_parquet(file_path: str, dtypes: dict) -> pd.DataFrame:
    df = pd.read_parquet(file_path)

    # Coerce columns to canonical types
    for column, target_dtype in dtypes.items():
        if column not in df.columns:
            df[column] = pd.NA
        try:
            if "float" in target_dtype:
                df[column] = pd.to_numeric(df[column], errors="coerce")
            elif "datetime" in target_dtype:
                df[column] = pd.to_datetime(df[column], errors="coerce")
            elif target_dtype == "boolean":
                df[column] = df[column].astype("boolean")
            else:
                df[column] = df[column].astype("string")
        except Exception as e:
            print(f"Warning: Failed to cast column {column} in {os.path.basename(file_path)}: {e}")
    
    # Ensure all canonical columns exist
    for col in dtypes:
        if col not in df.columns:
            df[col] = pd.NA

    return df[dtypes.keys()]

In [7]:
#handle duplicated cells
def load_and_normalize_parquet(file_path: str, dtypes: dict) -> pd.DataFrame:
    df = pd.read_parquet(file_path)

    # Coerce columns to canonical types
    for column, target_dtype in dtypes.items():
        if column not in df.columns:
            df[column] = pd.NA
        try:
            if "float" in target_dtype:
                df[column] = pd.to_numeric(df[column], errors="coerce")
            elif "datetime" in target_dtype:
                df[column] = pd.to_datetime(df[column], errors="coerce")
            elif target_dtype == "boolean":
                df[column] = df[column].astype("boolean")
            else:
                df[column] = df[column].astype("string")
        except Exception as e:
            print(f"Warning: Failed to cast column {column} in {os.path.basename(file_path)}: {e}")
    
    # Ensure all canonical columns exist
    for col in dtypes:
        if col not in df.columns:
            df[col] = pd.NA

    return df[dtypes.keys()]


In [8]:
consolidated_data = []

for file_name in parquet_files:
    file_path = os.path.join(data_dir, file_name)
    try:
        df = load_and_normalize_parquet(file_path, canonical_dtypes)
        df["Source_File"] = file_name  # Optional: keep source traceability
        consolidated_data.append(df)
    except Exception as e:
        print(f"Failed to process {file_name}: {e}")

# Final concatenated dataframe
unified_df = pd.concat(consolidated_data, ignore_index=True)


In [9]:
unified_df.head(2)

,Run_Date,Retailer,Retailer_Website,Store_Info,Category_Traversal,Brand,Product_Name,Product_ID,Universal_Product_ID,Variant_Info,Size_Or_Quantity,Number_Of_Reviews,Average_Rating,Original_Price,Sale_Price,Currency,Url,In_Stock,Source_File
0,2025-01-27 17:00:21,CubeSmart,cubesmart.com,"{""Store_Id"":""3222"",""Store_Name"":""CubeSmart of ...",CubeSmart Self Storage > Maryland Self Storage...,CubeSmart,Large,6efcc51c-879f-4914-bf62-7fe9e49a91d3,<NA>,"{""Features"":""Climate controlled, Elevator acce...",20'x10',NaN,NaN,437.0,262.2,USD,https://www.cubesmart.com/maryland-self-storag...,True,cubesmart_weekly_20250127_181344.parquet
1,2025-01-27 17:00:21,CubeSmart,cubesmart.com,"{""Store_Id"":""4245"",""Store_Name"":""CubeSmart of ...",CubeSmart Self Storage > Arizona Self Storage ...,CubeSmart,Large,6db3e08a-6122-46ff-b66f-eec08fdd5c3c,<NA>,"{""Features"":""Climate controlled, Elevator acce...",10'x20',NaN,NaN,273.0,163.8,USD,https://www.cubesmart.com/arizona-self-storage...,True,cubesmart_weekly_20250127_181344.parquet


## Data profiling

In [11]:
report = ProfileReport(df = unified_df, title = "data profiling report")
report.to_file("crisil_report.html")


C:\Users\Paola Otero\anaconda3\Lib\site-packages\ydata_profiling\profile_report.py:365: UserWarning: Try running command: 'pip install --upgrade Pillow' to avoid ValueError
  warnings.warn(


Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]


100%|██████████████████████████████████████████████████████████████████████████████████| 19/19 [00:03<00:00,  5.76it/s]


Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

No artists with labels found to put in legend.  Note that artists whose label start with an underscore are ignored when legend() is called with no argument.


Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

Export report to file:   0%|          | 0/1 [00:00<?, ?it/s]

# CRISIL Data Profiling Report – Key Insights

## 1. Data Quality Issues

### Missing Data
- `Number_Of_Reviews`: 100% missing
- `Average_Rating`: 100% missing
- `Universal_Product_ID`: 80.2% missing
- `Original_Price`: 12.1% missing

### Unsupported Types
- `Number_Of_Reviews` and `Average_Rating` are marked as **unsupported**, indicating potential issues with data type parsing or formatting. These fields may require cleaning or conversion.

### Constant Columns
- `Universal_Product_ID` has a constant empty value across all records.
- `Currency` has a constant value: `"USD"`.

---

## 2. High Correlation Between Features

The following features exhibit **high correlation**, which may introduce redundancy or multicollinearity if ML models are trained:

- `Brand` is highly correlated with:
  - `In_Stock`
  - `Retailer`
  - `Retailer_Website`
  - `Source_File`
  - `Product_Name`
  
- `In_Stock` shows the same correlation pattern as `Brand`.
- `Original_Price` is highly correlated with `Sale_Price`.
- `Retailer_Website` and `Source_File` are also correlated with `Brand` and its associated fields.

---

## 3. Dataset Summary

- **Number of Observations**: 99,595
- **Number of Variables**: 19
- **Missing Cells**: 291,117 (15.4%)
- **Duplicate Rows**: 0 (0.0%)
- **Total Memory Usage**: 13.9 MiB
- **Average Record Size**: 146.0 Bytes

---

## 4. Variable Type Breakdown

- **Numeric**: 2
- **Categorical**: 7
- **Text**: 6
- **Boolean**: 1
- **DateTime**: 1
- **Unsupported**: 2

---

## 5. Notes for Further Action

- Investigate the reason behind 100% missing values in `Number_Of_Reviews` and `Average_Rating`.
- Validate whether constant columns such as `Currency` and `Universal_Product_ID` add value to downstream tasks.
- Consider dimensionality reduction or feature selection to handle high correlation among multiple columns.
- Clean or transform unsupported columns to align with the expected data types.


## Field Criticality and Data Quality Constraints

The following analysis classifies each field in the dataset according to its role in supporting business operations, analytical tasks, and machine learning models, particularly for self-storage product offerings. Each field is evaluated based on:

- **Operational Criticality**: Importance for core business processes such as inventory management and availability tracking.
- **Analytical Value**: Usefulness for descriptive analytics, reporting, and exploration.
- **Modeling Impact**: Influence on predictive models, especially recommendation systems or pricing algorithms.

The **criticality matrix** establishes a foundation for defining data quality standards. Based on this classification, a set of **field-level constraints** has been specified to ensure that the most important fields meet minimum data completeness and validity thresholds. These constraints aim to safeguard downstream analytical reliability and model performance in production contexts.

## Criticality Matrix

| Field                 | Operational Criticality | Analytical Value | Modeling Impact | Notes |
|----------------------|--------------------------|------------------|------------------|-------|
| Run_Date             | High                     | High             | High             | Useful for tracking updates and time-based trends. |
| Retailer             | High                     | High             | High             | Essential for brand-level grouping and filtering. |
| Retailer_Website     | Medium                   | Low              | Low              | Mostly metadata; may support traceability. |
| Store_Info           | High                     | High             | Medium           | Contains location metadata critical for routing and geo-segmentation. |
| Category_Traversal   | Medium                   | High             | Medium           | Useful for categorization and taxonomy-level recommendations. |
| Brand                | High                     | Medium           | Medium           | Redundant with Retailer in this case, but still relevant for display and filters. |
| Product_Name         | High                     | Medium           | Medium           | Describes the unit; may help enrich feature sets. |
| Product_ID           | High                     | Medium           | High             | Unique identifier, required for joins, matching, and recommendations. |
| Universal_Product_ID | Low (currently)          | Low              | Low              | High null rate; not currently usable. |
| Variant_Info         | Medium                   | High             | Medium           | Holds detailed features; may need parsing but valuable in modeling. |
| Size_Or_Quantity     | High                     | High             | High             | Core attribute used in search filters and pricing models. |
| Number_Of_Reviews    | Low (currently)          | High             | High             | 100% missing but critical for trust and ranking models if populated. |
| Average_Rating       | Low (currently)          | High             | High             | Similar to above; highly influential in recommendations. |
| Original_Price       | High                     | High             | High             | Used in pricing logic and value-based modeling. |
| Sale_Price           | High                     | High             | High             | Represents the effective price; essential for promotion tracking and optimization. |
| Currency             | Medium                   | Low              | Low              | Constant field (USD); may be dropped if invariant. |
| Url                  | Medium                   | Low              | Low              | Useful for traceability or display but not for modeling. |
| In_Stock             | High                     | High             | High             | Core operational signal; likely to influence availability-based ranking. |

## Updated Data Constraints for Critical Features

| Field              | Constraint Type            | Threshold / Range                    | Justification |
|-------------------|----------------------------|--------------------------------------|---------------|
| Run_Date           | Missing Values             | ≤ 0.5%                               | Required for temporal freshness tracking and data versioning. |
| Retailer           | Missing Values             | ≤ 0.5%                               | Drives brand-level aggregation and analytics. |
| Store_Info         | JSON Parse Success         | 100% valid JSON                      | Structured parsing is necessary for location and feature extraction. |
| Category_Traversal | Missing Values             | ≤ 2%                                 | Important for taxonomy-based grouping; moderate tolerance. |
| Brand              | Missing Values             | ≤ 1%                                 | Critical for display, filtering, and modeling redundancy with Retailer. |
| Product_Name       | Missing Values             | ≤ 1%                                 | Needed for customer-facing naming logic and modeling context. |
| Product_ID         | Uniqueness                 | 100% unique per retailer             | Acts as the unique key across data systems and model features. |
| Variant_Info       | JSON Parse Success         | ≥ 95% valid                          | Enables extraction of product features; partial tolerance allowed. |
| Size_Or_Quantity   | Missing Values             | ≤ 1%                                 | Crucial for dimensional categorization and pricing. |
| Original_Price     | Missing Values             | ≤ 2%                                 | Required for discount logic and price sensitivity modeling. |
|                    | Min/Max Value              | ≥ 5 and ≤ 5760 (with 20% tolerance)  | Observed range: 5–4800; tolerance allows for future pricing shifts. |
| Sale_Price         | Missing Values             | ≤ 2%                                 | Core business metric; used in ranking and value-based recommendation. |
|                    | Min/Max Value              | ≥ 3 and ≤ 9644.4 (with 20% tolerance)| Observed range: 3–8037; tolerance accounts for future data packages. |
| In_Stock           | Missing Values             | ≤ 0.5%                               | Required for availability filtering and operational logic. |
